In [18]:
# I wanted to ask this as a question on Stackoverflow

backend_data = {
    "admins": ["Leo", "Martin", "Thomas", "Katrin"],
    "members": [
        "Leo",
        "Martin",
        "Thomas",
        "Katrin",
        "Subhayan",
        "Clemens",
        "Thoralf"
    ],
    "juniors": ["Orianne", "Antonia", "Sarah"]
}

class Backend:
    def __init__(self, data):
        self.backend_data = data
        
    def get_all_admins(self, extras=None):
        if extras is None:
            return self.backend_data.get("admins")
        data = self.backend_data.get("admins")
        data.extend(extras)
        return data
    
    def get_all_members(self):
        return self.backend_data.get("members")
    
    def get_all_juniors(self):
        return self.backend_data.get("juniors")
    
    
class BackendAdaptor:
    # Does some conversion and validation
    # For the methods get_all_juniors and get_all_admins
    # The method just delegates to backend and returns a generator of the results
    # While for the method get_all_members there is some validation
    def __init__(self, backend):
        self.backend = backend
        
#     def get_all_admins(self, extras=None):
#         return (admin for admin in self.backend.get_all_admins(extras))
    
    def get_all_members(self):
        return (member for member in self.backend.get_all_members() if member not in self.backend.get_all_admins())
    
#     def get_all_juniors(self):
#         return (junior for junior in self.backend.get_all_juniors())
    
    def __getattr__(self, name):
        if not hasattr(self.backend, name):
            raise AttributeError(f"'{name}' not in backend.")
        return lambda *args, **kwargs: (i for i in getattr(self.backend, name)(*args, **kwargs))
    
    
if __name__ == "__main__":
    backend = Backend(data=backend_data)
    adaptor = BackendAdaptor(backend=backend)
    print(f"All admins are : {list(adaptor.get_all_admins(extras=['Gregor', 'Alex']))}")
    print(f"All members are : {list(adaptor.get_all_members())}")
    print(f"All juniors are : {list(adaptor.get_all_juniors())}")
    adaptor.get_something_else() # This should raise an attribute error
        

All admins are : ['Leo', 'Martin', 'Thomas', 'Katrin', 'Gregor', 'Alex']
All members are : ['Subhayan', 'Clemens', 'Thoralf']
All juniors are : ['Orianne', 'Antonia', 'Sarah']


AttributeError: 'get_something_else' not in backend.

In [3]:
numbers = list(range(10, 100, 5))

In [4]:
numbers

[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]

In [11]:
some_indices = [3, 6, 10, 13]
start_index = 0
for index in some_indices:
    print(f"Number at index : {numbers[index]}")
    print(numbers[start_index:index])
    start_index = index + 1
print(f"Finally : {numbers[start_index:]}")

Number at index : 25
[10, 15, 20]
Number at index : 40
[30, 35]
Number at index : 60
[45, 50, 55]
Number at index : 75
[65, 70]
Finally : [80, 85, 90, 95]


In [3]:
# Recipe to break a dataframe into chunks
# I must have used this in my expense viewer project

import itertools

def break_up_iterable_into_chunks(numbers, chunk_indices):
    start_index = 0
    for index in chunk_indices:
        yield numbers[start_index:index]
        start_index = index + 1
    yield numbers[start_index:]
    

for chunk in itertools.islice(break_up_iterable_into_chunks(numbers=list(range(10, 100, 5)), chunk_indices=[3, 6, 10, 13]), 1, None):
    print(chunk)

[30, 35]
[45, 50, 55]
[65, 70]
[80, 85, 90, 95]


In [16]:
# Very very trivial order class  just to check
# how a property factory works

def quantity(storage):
    def qty_getter(instance):
        print("Getter function getting called")
        return instance.__dict__[storage]
    
    def qty_setter(instance, value):
        print("Setter function getting called")
        if value < 0:
            raise ValueError("Value should be more than 0")
        else:
            instance.__dict__[storage] = value
            
    return property(qty_getter, qty_setter)

class Order:
    items = quantity('items')
    discount = quantity('discount')
    
    def __init__(self, name, items, discount):
        self.name = name
        self.items = items
        self.discount = discount
        
    def calculate_total(self):
        # Lets assume that the prince of the item is 100 
        return (self.items * (100 - self.discount))
        
    

In [17]:
c = Order("Vegetables", 20, 10)

Setter function getting called
Setter function getting called


In [18]:
c.calculate_total()

Getter function getting called
Getter function getting called


1800

In [19]:
c.items = -10

Setter function getting called


ValueError: Value should be more than 0

In [20]:
c.discount = -20

Setter function getting called


ValueError: Value should be more than 0

# using descriptors(Changed the example a bit from Fluent Python, using PEP 487)

In [6]:
class Quantity:
    def __set_name__(self, class_name, name):
        print(f"Setting the {name} attribute for {class_name}")
        self.name = name
        
    def __get__(self, instance, class_name=None):
        print(f"Checking the class name inside the get method {class_name}")
        return instance.__dict__[self.name]
        
        
    def __set__(self, instance, value):
        if value < 0 or value > 100:
            raise ValueError(f"{value} should be between 0 and 100")
        instance.__dict__[self.name] = value
        
    
class Order:
    # Data descriptors
    items = Quantity()
    discount = Quantity()
    
    def __init__(self, name, items, discount):
        self.name = name
        self.items = items
        self.discount = discount
        
    def calculate_total(self):
        # Lets assume that the prince of the item is 100 
        return (self.items * (100 - self.discount))

Setting the items attribute for <class '__main__.Order'>
Setting the discount attribute for <class '__main__.Order'>


In [3]:
o = Order("Fridge", 10, 20)

In [4]:
o.calculate_total()

Checking the class name inside the get method <class '__main__.Order'>
Checking the class name inside the get method <class '__main__.Order'>


800

In [7]:
new = Order("Refrigerator", -20, 100)

ValueError: -20 should be between 0 and 100

In [9]:
# Check if we can assign something to the class attribute
Order.items = 100 # I think this is completly replacing the descriptor instance with something else entirely

In [10]:
Order.items

100

In [3]:
class Vehicle():
    can_fly = False
    number_of_weels = 0

class Car(Vehicle):
    number_of_weels = 4

    def __init__(self, color):
        self.color = color

my_car = Car("red")
print(my_car.__dict__)
print(type(my_car).__dict__)


{'color': 'red'}
{'__module__': '__main__', 'number_of_weels': 4, '__init__': <function Car.__init__ at 0x7f815fc53940>, '__doc__': None}


In [6]:
print(Vehicle.__dict__)

{'__module__': '__main__', 'can_fly': False, 'number_of_weels': 0, '__dict__': <attribute '__dict__' of 'Vehicle' objects>, '__weakref__': <attribute '__weakref__' of 'Vehicle' objects>, '__doc__': None}


In [8]:
class Person:
    def __init__(self, name):
        self.name = name

In [9]:
p = Person(name="Subhayan")

In [10]:
Person.age = 24


In [11]:
p.age

24

In [12]:
dir(Person)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'age']

In [44]:
from datetime import datetime

class Person:
    def __init__(self, name, year_of_birth):
        self.name = name
        self.year_of_birth = year_of_birth
        self.age = datetime.now().year - self.year_of_birth
        
        
    @property
    def age(self):
        print("Accessing the age from getter function")
        return self._age
    
    @age.setter
    def age(self, value):
        if value <= 0 or value > 100:
            raise ValueError(f"{value} cannot be the value of age")
        self._age = value
        
    @classmethod
    def create(cls, name, date_of_birth):
        try:
            year_of_birth_obj = datetime.strptime(date_of_birth,'%Y-%m-%d')
            year_of_birth = year_of_birth_obj.year
        except:
            raise ValueError("Please provide input date in the following format YYYY-MM-DD")
        
        return cls(name=name, year_of_birth=year_of_birth)

In [45]:
p = Person.create('Poulomi', '1988-01-13')

In [46]:
p.age = -120

ValueError: -120 cannot be the value of age

In [31]:
p = Person.create('Subhayan', '2020-12-15')

0


ValueError: 0 cannot be the value of age